---
title: Access Internet from Private Subnet in VPC
author: Ziyue Li
categories:
- AWS
- Network
date: 2022-11-23
description: How to setup AWS VPC so we can access internet from the private subnet.
image: images/VPC-access_internet_from_private_subnet.png 
layout: post
execute:
    echo: true
    eval: false
    freeze: true
---

![](images/VPC-access_internet_from_private_subnet.png){fig-align="center"}

I'm finally forced to learn the network configuration stuff in AWS VPC.
The thing that was confusing me was the NAT Gateway.

For a private subnet to access the internet, we route traffic from the private subnet through a NAT Gateway which keeps out all traffic initiated from the internet.
This gateway needs to be in a public subnet, so that it can access the internet through the Internet Gateway.

To route the traffic in this way, we need to make sure there's an entry in the route table for the public subnet to connect to the Internet Gateway, and there's an entry in the route table for the private subnet to connect to the NAT Gateway.

- **Step 1**: [Create VPC, Public and Private Subnets](https://docs.aws.amazon.com/vpc/latest/userguide/working-with-vpcs.html#create-vpc-and-other-resources)

    Create public and private subnets for each availability zone. Make sure that the subnets don't overlap. For an explanation of subnets and CIDR notations:

    {{< youtube s_Ntt6eTn94 >}}

- **Step 2**: Create [Internet Gateway](https://docs.aws.amazon.com/vpc/latest/userguide/VPC_Internet_Gateway.html#Add_IGW_Attach_Gateway) and [NAT Gateway](https://docs.aws.amazon.com/vpc/latest/userguide/vpc-nat-gateway.html#nat-gateway-creating)

    Make sure to create the NAT Gateway for each availability zone, and make sure they are created in the **public** subnets, select "public" connectivity type, and allocate an elastic IP Address.

- **Step 3**: Create [Route Tables](https://docs.aws.amazon.com/vpc/latest/userguide/VPC_Route_Tables.html)

    For each public subnet, create a route table, add an entry to connect to the internet gateway, and associate it to that subnet.

    For each private subnet, create a route table, add an entry to connect to the NAT Gateway in the same availability zone, and associate it to that private subnet.

- **Step 4**: Add access to AWS resources

    If we want the subnets to have access to AWS services such as S3, we can [create an endpoint for AWS services](https://docs.aws.amazon.com/vpc/latest/privatelink/vpc-endpoints-s3.html#create-gateway-endpoint-s3), and associate that endpoint to the route tables we created above.

- **Step 5**: Make Sure [Network ACL](https://docs.aws.amazon.com/vpc/latest/userguide/vpc-network-acls.html) and [Security Groups](https://docs.aws.amazon.com/vpc/latest/userguide/VPC_SecurityGroups.html) Don't Block the Internet Traffic

    When selecting security groups to use for services launched in the private subnet, make sure it allows traffic from and to the NAT Gateway. 
    
    **I was tripped up by this when setting up network connections for EMR Serverless.**

{{< tweet curiosity_notes 1595508141554339882 >}}